In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score
)
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

csv_path = "/content/drive/MyDrive/MS/1st Sem/NLP/Data/depression_dataset_reddit_cleaned.csv"

df = pd.read_csv(csv_path)

df.head()

df.columns

In [ ]:
text_col  = "clean_text"
label_col = "is_depression"

df = df[[text_col, label_col]].dropna()

df.rename(columns={text_col: "text", label_col: "label"}, inplace=True)

df.head()

In [ ]:
df["label"].value_counts()

In [ ]:

label_map = {
    "control": 0,
    "depression": 1,
    "depressed": 1,
    "non-depressed": 0
}

if df["label"].dtype == "object":
    df["label"] = df["label"].map(label_map)

df["label"].value_counts()

In [ ]:
plt.figure(figsize=(4,4))
df["label"].value_counts().plot(kind="bar")
plt.xticks([0,1], ["non-depressed (0)", "depressed (1)"], rotation=0)
plt.title("Label distribution")
plt.show()

In [ ]:
df["text"] = df["text"].astype(str).str.strip()
df = df[df["text"].str.len() > 0].reset_index(drop=True)

len(df), df.head(3)

In [ ]:
!pip -q install wordcloud

import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS

In [ ]:
# All depressed posts (label == 1)
depressed_text = " ".join(df[df["label"] == 1]["text"].astype(str).tolist())

wc_depressed = WordCloud(
    width=1200,
    height=800,
    background_color="white",
    stopwords=STOPWORDS,
    max_words=200,
    collocations=True
).generate(depressed_text)

plt.figure(figsize=(10, 6))
plt.imshow(wc_depressed, interpolation="bilinear")
plt.axis("off")
plt.title("Word Cloud – Depressed Posts", fontsize=16)
plt.show()

In [ ]:
# All non-depressed posts (label == 0)
non_depressed_text = " ".join(df[df["label"] == 0]["text"].astype(str).tolist())

wc_non_dep = WordCloud(
    width=1200,
    height=800,
    background_color="white",
    stopwords=STOPWORDS,
    max_words=200,
    collocations=True
).generate(non_depressed_text)

plt.figure(figsize=(10, 6))
plt.imshow(wc_non_dep, interpolation="bilinear")
plt.axis("off")
plt.title("Word Cloud – Non-depressed Posts", fontsize=16)
plt.show()

In [ ]:
extra_stopwords = {
    "im", "ive", "dont", "cant", "didnt", "doesnt", "really",
    "like", "just", "feel", "feeling", "feelings",
    "reddit", "people", "one", "get", "got"
}

stopwords = STOPWORDS.union(extra_stopwords)

In [ ]:
X = df["text"].values
y = df["label"].values

# First: train + temp (val+test)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Second: split temp into val + test
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

len(X_train), len(X_val), len(X_test)

In [ ]:


tfidf = TfidfVectorizer(
    max_features=30000,
    ngram_range=(1,2),
    min_df=2,
    max_df=0.95
)

X_train_tfidf = tfidf.fit_transform(X_train)
X_val_tfidf   = tfidf.transform(X_val)
X_test_tfidf  = tfidf.transform(X_test)

X_train_tfidf.shape, X_val_tfidf.shape, X_test_tfidf.shape

In [ ]:
log_reg = LogisticRegression(
    max_iter=1000,
    class_weight="balanced",
    n_jobs=-1
)

log_reg.fit(X_train_tfidf, y_train)

In [ ]:
y_val_pred = log_reg.predict(X_val_tfidf)

print("Validation accuracy:", accuracy_score(y_val, y_val_pred))
print("\nValidation report:\n")
print(classification_report(y_val, y_val_pred))

In [ ]:
y_test_pred = log_reg.predict(X_test_tfidf)

print("Test accuracy:", accuracy_score(y_test, y_test_pred))
print("\nTest report:\n")
print(classification_report(y_test, y_test_pred))

In [ ]:
cm = confusion_matrix(y_test, y_test_pred)

plt.figure(figsize=(4,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=["non-depressed", "depressed"],
            yticklabels=["non-depressed", "depressed"])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("LogReg + TF-IDF — Test Confusion Matrix")
plt.show()

In [ ]:
!pip -q install transformers datasets accelerate evaluate

import pandas as pd
from sklearn.model_selection import train_test_split

# If your columns are named differently, change here:
TEXT_COL = "text"
LABEL_COL = "label"

# Make sure labels are int
df[LABEL_COL] = df[LABEL_COL].astype(int)

train_df, test_df = train_test_split(
    df[[TEXT_COL, LABEL_COL]],
    test_size=0.2,
    random_state=42,
    stratify=df[LABEL_COL]
)

len(train_df), len(test_df)

In [ ]:
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Wrap into HF Dataset objects
train_ds = Dataset.from_pandas(train_df.reset_index(drop=True))
test_ds  = Dataset.from_pandas(test_df.reset_index(drop=True))

def tokenize_batch(batch):
    return tokenizer(
        batch[TEXT_COL],
        truncation=True,
        padding="max_length",
        max_length=128
    )

train_ds = train_ds.map(tokenize_batch, batched=True)
test_ds  = test_ds.map(tokenize_batch, batched=True)

# Tell HF which column is the label
train_ds = train_ds.rename_column(LABEL_COL, "labels")
test_ds  = test_ds.rename_column(LABEL_COL, "labels")

train_ds.set_format("torch")
test_ds.set_format("torch")

In [ ]:
import evaluate
import numpy as np
from transformers import (
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
)

num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

accuracy_metric  = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric    = evaluate.load("recall")
f1_metric        = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)

    acc = accuracy_metric.compute(predictions=preds, references=labels)
    prec = precision_metric.compute(predictions=preds, references=labels, average="macro")
    rec = recall_metric.compute(predictions=preds, references=labels, average="macro")
    f1 = f1_metric.compute(predictions=preds, references=labels, average="macro")

    return {
        "accuracy": acc["accuracy"],
        "precision": prec["precision"],
        "recall": rec["recall"],
        "f1": f1["f1"],
    }

training_args = TrainingArguments(
    output_dir="./distilbert-depression",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
train_result = trainer.train()
trainer.save_model("./distilbert-depression-best")

train_result.metrics

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import torch

# Get logits for the test set
pred_output = trainer.predict(test_ds)
logits = pred_output.predictions
y_test = pred_output.label_ids
y_pred = np.argmax(logits, axis=-1)

print("Test metrics from Trainer:", pred_output.metrics)

print("\nSklearn classification report (DistilBERT):\n")
print(classification_report(y_test, y_pred, digits=4))

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=["non-depressed","depressed"],
            yticklabels=["non-depressed","depressed"])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("DistilBERT — Test Confusion Matrix")
plt.show()

In [ ]:
df_errors = pd.DataFrame({
    "text": test_df["text"].values,   # original text
    "true": y_test,                   # true labels from trainer.predict
    "pred": y_pred                    # predicted labels
})

df_errors.head()

In [ ]:
wrong_df = df_errors[df_errors["true"] != df_errors["pred"]]
print("Total misclassified samples:", len(wrong_df))
wrong_df.head(10)

In [ ]:
false_positives = df_errors[(df_errors["true"] == 0) & (df_errors["pred"] == 1)]
print("False positives:", len(false_positives))
false_positives.head(5)

In [ ]:
false_negatives = df_errors[(df_errors["true"] == 1) & (df_errors["pred"] == 0)]
print("False negatives:", len(false_negatives))
false_negatives.head(5)

In [ ]:
# Compute probabilities using softmax
probs = torch.softmax(torch.tensor(logits), dim=1).numpy()
conf = probs.max(axis=1)

df_errors["confidence"] = conf

high_conf_wrong = df_errors[(df_errors["true"] != df_errors["pred"]) &
                            (df_errors["confidence"] > 0.90)]

print("High-confidence wrong predictions:", len(high_conf_wrong))
high_conf_wrong.head(10)

In [ ]:
import matplotlib.pyplot as plt

counts = {
    "Misclassified": len(wrong_df),
    "False Positives": len(false_positives),
    "False Negatives": len(false_negatives),
    "High-Confidence Errors": len(high_conf_wrong)
}

plt.figure(figsize=(8,5))
plt.bar(counts.keys(), counts.values(), color=["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728"])
plt.title("Error Analysis Summary", fontsize=14)
plt.xlabel("Error Type", fontsize=12)
plt.ylabel("Count", fontsize=12)
plt.xticks(rotation=20)
plt.grid(axis="y", linestyle="--", alpha=0.6)
plt.show()

In [ ]:
# === Sarcasm dataset: load & clean ===
import pandas as pd
from sklearn.model_selection import train_test_split
from google.colab import drive
drive.mount('/content/drive')


sarc_csv_path  = "/content/drive/MyDrive/MS/1st Sem/NLP/Data/train-balanced-sarcasm.csv"
SARC_TEXT_COL  = "comment"    # e.g. "headline", "comment", "text"
SARC_LABEL_COL = "label"      # e.g. 0/1 or "sarcasm"

df_sarc = pd.read_csv(sarc_csv_path)

df_sarc = df_sarc[[SARC_TEXT_COL, SARC_LABEL_COL]].dropna()
df_sarc[SARC_TEXT_COL] = df_sarc[SARC_TEXT_COL].astype(str).str.strip()

if df_sarc[SARC_LABEL_COL].dtype == "object":
    sarc_map = {
        "sarcasm": 1,
        "sarcastic": 1,
        "not_sarcasm": 0,
        "non_sarcastic": 0,
        "normal": 0
    }
    df_sarc[SARC_LABEL_COL] = df_sarc[SARC_LABEL_COL].map(sarc_map)

df_sarc = df_sarc.dropna(subset=[SARC_LABEL_COL])
df_sarc[SARC_LABEL_COL] = df_sarc[SARC_LABEL_COL].astype(int)

print(df_sarc.head())
print(df_sarc[SARC_LABEL_COL].value_counts())

In [ ]:
# === Train / test split for sarcasm ===
train_sarc_df, test_sarc_df = train_test_split(
    df_sarc[[SARC_TEXT_COL, SARC_LABEL_COL]],
    test_size=0.2,
    random_state=42,
    stratify=df_sarc[SARC_LABEL_COL]
)

len(train_sarc_df), len(test_sarc_df)

In [ ]:
# === Hugging Face Dataset + tokenization for sarcasm ===
from datasets import Dataset
from transformers import AutoTokenizer

sarc_model_name = "distilbert-base-uncased"
tokenizer_sarc = AutoTokenizer.from_pretrained(sarc_model_name)

train_sarc_ds = Dataset.from_pandas(train_sarc_df.reset_index(drop=True))
test_sarc_ds  = Dataset.from_pandas(test_sarc_df.reset_index(drop=True))

def tokenize_sarc(batch):
    return tokenizer_sarc(
        batch[SARC_TEXT_COL],
        truncation=True,
        padding="max_length",
        max_length=128
    )

train_sarc_ds = train_sarc_ds.map(tokenize_sarc, batched=True)
test_sarc_ds  = test_sarc_ds.map(tokenize_sarc, batched=True)

train_sarc_ds = train_sarc_ds.rename_column(SARC_LABEL_COL, "labels")
test_sarc_ds  = test_sarc_ds.rename_column(SARC_LABEL_COL, "labels")

# If your DF had an index col carried over, you can drop it if needed:
for col in ["__index_level_0__", "index"]:
    if col in train_sarc_ds.column_names:
        train_sarc_ds = train_sarc_ds.remove_columns([col])
    if col in test_sarc_ds.column_names:
        test_sarc_ds = test_sarc_ds.remove_columns([col])

train_sarc_ds.set_format("torch")
test_sarc_ds.set_format("torch")

train_sarc_ds, test_sarc_ds

In [ ]:
# === Model, metrics, Trainer for sarcasm ===
import numpy as np
!pip -q install transformers datasets accelerate evaluate
import evaluate
from transformers import (
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
)

num_labels_sarc = 2
model_sarc = AutoModelForSequenceClassification.from_pretrained(
    sarc_model_name,
    num_labels=num_labels_sarc
)

data_collator_sarc = DataCollatorWithPadding(tokenizer=tokenizer_sarc)

accuracy_metric  = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric    = evaluate.load("recall")
f1_metric        = evaluate.load("f1")

def compute_metrics_sarc(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)

    acc = accuracy_metric.compute(predictions=preds, references=labels)
    prec = precision_metric.compute(predictions=preds, references=labels, average="macro")
    rec = recall_metric.compute(predictions=preds, references=labels, average="macro")
    f1 = f1_metric.compute(predictions=preds, references=labels, average="macro")

    return {
        "accuracy": acc["accuracy"],
        "precision": prec["precision"],
        "recall": rec["recall"],
        "f1": f1["f1"],
    }

training_args_sarc = TrainingArguments(
    output_dir="./distilbert-sarcasm",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none"
)

trainer_sarc = Trainer(
    model=model_sarc,
    args=training_args_sarc,
    train_dataset=train_sarc_ds,
    eval_dataset=test_sarc_ds,
    tokenizer=tokenizer_sarc,
    data_collator=data_collator_sarc,
    compute_metrics=compute_metrics_sarc,
)

In [ ]:
# === Train sarcasm model ===
sarc_train_result = trainer_sarc.train()
trainer_sarc.save_model("./distilbert-sarcasm-best")

sarc_train_result.metrics

In [ ]:
# === Evaluate sarcasm model on its test set ===
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

sarc_pred_output = trainer_sarc.predict(test_sarc_ds)
sarc_logits = sarc_pred_output.predictions
sarc_y_true = sarc_pred_output.label_ids
sarc_y_pred = np.argmax(sarc_logits, axis=-1)

print("Sarcasm test metrics:", sarc_pred_output.metrics)
print("\nSarcasm classification report:\n")
print(classification_report(sarc_y_true, sarc_y_pred, digits=4))

cm_sarc = confusion_matrix(sarc_y_true, sarc_y_pred)
plt.figure(figsize=(5,4))
sns.heatmap(cm_sarc, annot=True, fmt="d", cmap="Purples",
            xticklabels=["non-sarcastic","sarcastic"],
            yticklabels=["non-sarcastic","sarcastic"])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Sarcasm DistilBERT — Test Confusion Matrix")
plt.show()

In [ ]:
!pip -q install gradio

In [ ]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import gradio as gr
from google.colab import drive
drive.mount('/content/drive')


MODEL_DEP_DIR = "./distilbert-depression-best"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dep_tokenizer = AutoTokenizer.from_pretrained(MODEL_DEP_DIR)
dep_model = AutoModelForSequenceClassification.from_pretrained(MODEL_DEP_DIR)
dep_model.to(device)
dep_model.eval()

dep_id2label = {0: "non-depressed", 1: "depressed"}

In [ ]:
def classify_depression(text: str):
    text = text.strip()
    if not text:
        return "Please enter some text.", 0.0

    enc = dep_tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=128,
        return_tensors="pt"
    )
    input_ids = enc["input_ids"].to(device)
    attention_mask = enc["attention_mask"].to(device)

    with torch.no_grad():
        logits = dep_model(input_ids, attention_mask=attention_mask).logits
        probs = torch.softmax(logits, dim=-1).cpu().numpy()[0]

    pred_id = int(np.argmax(probs))
    pred_label = dep_id2label[pred_id]
    confidence = float(probs[pred_id])
    return pred_label, round(confidence, 4)

In [ ]:
MODEL_SARC_DIR = "./distilbert-sarcasm-best"

sarc_tokenizer = AutoTokenizer.from_pretrained(MODEL_SARC_DIR)
sarc_model = AutoModelForSequenceClassification.from_pretrained(MODEL_SARC_DIR)
sarc_model.to(device)
sarc_model.eval()

# assuming 0 = non-sarcastic, 1 = sarcastic
sarc_id2label = {0: "non-sarcastic", 1: "sarcastic"}

In [ ]:
def classify_sarcasm(text: str):
    text = text.strip()
    if not text:
        return "Please enter some text.", 0.0

    enc = sarc_tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=128,
        return_tensors="pt"
    )
    input_ids = enc["input_ids"].to(device)
    attention_mask = enc["attention_mask"].to(device)

    with torch.no_grad():
        logits = sarc_model(input_ids, attention_mask=attention_mask).logits
        probs = torch.softmax(logits, dim=-1).cpu().numpy()[0]

    pred_id = int(np.argmax(probs))
    pred_label = sarc_id2label[pred_id]
    confidence = float(probs[pred_id])
    return pred_label, round(confidence, 4)

In [ ]:
def analyze_post(text: str):
    text = text.strip()
    if not text:
        return (
            "—",
            0.0,
            "—",
            0.0,
            "Please enter some text above to get a prediction.",
        )

    dep_label, dep_conf = classify_depression(text)
    sarc_label, sarc_conf = classify_sarcasm(text)

    # Simple interpretation logic
    msg_lines = []
    msg_lines.append(f"- **Depression model**: `{dep_label}` (confidence ≈ {dep_conf:.2f})")
    msg_lines.append(f"- **Sarcasm model**: `{sarc_label}` (confidence ≈ {sarc_conf:.2f})")

    # A few informal rules
    if dep_label == "depressed" and dep_conf > 0.75 and sarc_conf < 0.6:
        msg_lines.append(
            "\n⚠️ Model sees this as **likely depressed** and not strongly sarcastic. "
            "In a real system this would deserve attention."
        )
    elif dep_label == "depressed" and sarc_conf >= 0.6:
        msg_lines.append(
            "\n🤔 Model thinks the content is **depressed**, but also **highly sarcastic**. "
            "The emotional signal might be ironic or playful, so this prediction should be treated with caution."
        )
    elif dep_label == "non-depressed" and sarc_conf >= 0.6:
        msg_lines.append(
            "\n🙂 Overall tone looks **non-depressed**, but sarcasm is high – "
            "this could be playful joking rather than genuine distress."
        )
    else:
        msg_lines.append(
            "\n✅ Model sees this as **non-depressed** with low sarcasm. "
            "Tone looks mostly neutral or positive."
        )

    interpretation = "\n".join(msg_lines)
    return dep_label, round(dep_conf, 4), sarc_label, round(sarc_conf, 4), interpretation

In [ ]:
theme = gr.themes.Soft(primary_hue="orange", neutral_hue="slate")

with gr.Blocks(theme=theme) as demo:
    gr.Markdown(
        """
# Reddit Depression + Sarcasm Classifier

Demo of DistilBERT **depression classifier** plus a separate **sarcasm detector**.

- Depression model output: `depressed` vs `non-depressed`
- Sarcasm model output: `sarcastic` vs `non-sarcastic`
- Interpretation combines both scores.

⚠️ **Research prototype only – not a clinical or diagnostic tool.**
        """
    )

    with gr.Row():
        # Left: input + examples
        with gr.Column(scale=1):
            txt = gr.Textbox(
                lines=7,
                label="Reddit post / text",
                placeholder="Paste a Reddit-style post here…",
            )

            gr.Examples(
                examples=[
                    ["I’m so done with life, nothing feels worth it anymore."],
                    ["Had a great day at work, feeling proud of myself!"],
                    ["Yeah, everything is *totally* fine while I cry myself to sleep 😂"],
                    ["Another Monday, another meeting… living the dream 🙃"],
                ],
                inputs=txt,
            )

        # Right: outputs
        with gr.Column(scale=1):
            gr.Markdown("### 🔍 Model predictions")

            with gr.Row():
                dep_label_out = gr.Label(label="Depression prediction")
                sarc_label_out = gr.Label(label="Sarcasm prediction")

            with gr.Row():
                dep_conf_out = gr.Slider(
                    0, 1, value=0, step=0.01, interactive=False,
                    label="Depression confidence (0–1)"
                )
                sarc_conf_out = gr.Slider(
                    0, 1, value=0, step=0.01, interactive=False,
                    label="Sarcasm confidence (0–1)"
                )

            #gr.Markdown("### 🧠 Combined interpretation")
            #interp_md = gr.Markdown(value="Model explanation will appear here.")

            with gr.Row():
                btn_run = gr.Button("Submit", variant="primary")
                btn_clear = gr.Button("Clear")

    # Info / how-to section
    with gr.Accordion("How to read these scores", open=False):
        gr.Markdown(
            """
- **Confidence** is the model's probability for the predicted label.
- High depression score + low sarcasm score → more reliable depression signal.
- High depression score + high sarcasm score → text might be joking/ironic.
- This app is meant to explore model behavior, **not** to make clinical decisions.
            """
        )

    # Wire buttons
    btn_run.click(
        analyze_post,
        inputs=txt,
        outputs=[dep_label_out, dep_conf_out, sarc_label_out, sarc_conf_out, interp_md],
    )

    def clear_all():
        return "", "—", 0.0, "—", 0.0, "Cleared. Enter new text above."

    btn_clear.click(
        clear_all,
        inputs=None,
        outputs=[txt, dep_label_out, dep_conf_out, sarc_label_out, sarc_conf_out, interp_md],
    )

demo.launch(share=True)